# K-Means Simulation

In this notebook, since it is a pretty simple algorithm, I will try to calculate Kmeans from scratch. 

## Algorithm Explanation

Given some data, KMeans tries to find the most natural groups/clusters for this data based on their distance to K cluster centers. The algorithm updates the positions of these cluster centers iteratively by taking the mean of the members' positions until there is no change in the positions of these centers, and these groups are defined as the final assignments. 

The steps of operation can be defined as follows:

1. Define number of cluster center, K
2. Define initial K cluster centers, by selecting random values in the feature space.
3. Calculate the distance of all points to the cluster centers, and assign point to the nearest center.
4. Repeat: 

    a) Update the position of the cluster centers by taking the mean of their feature values of it's community members.
    
    b) Recalculate the distance between all data points and the new centers. 
    
    c) Stop when assignments/labels no longer change.
    
There are different ways to perform these operations, with multiple variations of the above. Here, I will calculate the minimal example for simplicity.

We'll also need data to sample, so to use an interesting one, let's use the CDC Nutrition Demographic data uploaded to [Kaggle](https://www.kaggle.com/cdc/national-health-and-nutrition-examination-survey). 

## Load Data and Preprocess

In [396]:
import pandas as pd

data= pd.read_csv('cdc_demographic.csv')
print(data.shape)
data.head()

(10175, 47)


,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,73557,8,2,1,69,NaN,4,4,1.0,NaN,...,3.0,4.0,NaN,13281.237386,13481.042095,1,112,4.0,4.0,0.84
1,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,3.0,1.0,1.0,23682.057386,24471.769625,1,108,7.0,7.0,1.78
2,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,4.0,1.0,3.0,57214.803319,57193.285376,1,109,10.0,10.0,4.51
3,73560,8,2,1,9,NaN,3,3,1.0,119.0,...,3.0,1.0,4.0,55201.178592,55766.512438,2,109,9.0,9.0,2.52
4,73561,8,2,2,73,NaN,3,3,1.0,NaN,...,5.0,1.0,5.0,63709.667069,65541.871229,2,116,15.0,15.0,5.00


As you can see, there are a few columns with lots of nans or nulls. Let's remove those columns if they exceed a threshold of 33%. For the remaining rows, we'll replace the nan with the median value for that column.

In [397]:
#Drop heavy-null columns
data= data.dropna(thresh=len(data)/3, axis=1)

#Replace remaining nulls
data= data.fillna(data.median())
print(data.shape)
data.head()

(10175, 42)


,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,73557,8,2,1,69,4,4,1.0,103.0,1.0,...,3.0,4.0,4.0,13281.237386,13481.042095,1,112,4.0,4.0,0.84
1,73558,8,2,1,54,3,3,1.0,103.0,2.0,...,3.0,1.0,1.0,23682.057386,24471.769625,1,108,7.0,7.0,1.78
2,73559,8,2,1,72,3,3,2.0,103.0,1.0,...,4.0,1.0,3.0,57214.803319,57193.285376,1,109,10.0,10.0,4.51
3,73560,8,2,1,9,3,3,1.0,119.0,2.0,...,3.0,1.0,4.0,55201.178592,55766.512438,2,109,9.0,9.0,2.52
4,73561,8,2,2,73,3,3,1.0,103.0,2.0,...,5.0,1.0,5.0,63709.667069,65541.871229,2,116,15.0,15.0,5.00


## Define Initial Cluster Centers

To do this, we'll need to essentially 1) take our data matrix of n columns, and column-wise, randomly sample the space to get a value from each of the columns. We'll do this K times for each of the K cluster centers.

In [398]:
def initial_centers(k_num, input_data):
    
    center_values= []
    
    for i in range(k_num):
        rand_values= [np.random.choice(input_data.iloc[:,i]) for i in range(input_data.shape[1])]
        center_values.append(rand_values)
        
    return center_values

In [399]:
centers0= pd.DataFrame(initial_centers(5, data), columns= data.columns)
print(centers0.shape)
centers0.head()

(5, 42)


,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,DMQMILIZ,...,DMDHREDU,DMDHRMAR,DMDHSEDU,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,76224,8,2,1,20,1,3,2.0,103.0,2.0,...,5.0,1.0,5.0,57206.653095,116084.743624,1,110,7.0,8.0,4.25
1,74108,8,2,1,67,1,6,1.0,103.0,2.0,...,5.0,1.0,4.0,43597.759376,18175.820698,1,107,7.0,13.0,3.81
2,74782,8,2,2,24,3,2,2.0,103.0,2.0,...,4.0,1.0,4.0,18100.496948,24850.493384,1,106,6.0,5.0,1.68
3,73816,8,2,1,80,4,4,2.0,103.0,2.0,...,3.0,2.0,1.0,109829.899489,104811.617948,1,110,9.0,4.0,4.19
4,82107,8,2,2,78,2,6,2.0,155.0,2.0,...,4.0,5.0,4.0,18184.520132,8382.423146,1,111,8.0,15.0,1.37


## Calculate the Distance to Centers
Since not all features are scaled the same way, we need to normalize them so that features with large magnitudes don't dominate algorithm.

In [400]:
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler

def preproc_for_dist(array_1, array_2):
    """This function normalizes the data before calculating distances of arrays."""
    #Scale data before calculating distance
    scaler = MinMaxScaler()
    scaler.fit(array_1)
     
    norm_1= scaler.transform(array_1)
    norm_2= scaler.transform(array_2)
    
    return norm_1, norm_2

In [401]:
def calc_dist_two_arrays(my_array_1, my_array_2):
    norm_array_1, norm_array_2= preproc_for_dist(my_array_1, my_array_2)
    dist_matrix= pd.DataFrame(distance.cdist(norm_array_1, norm_array_2, metric='euclidean'))
    
    return dist_matrix

In [402]:
def calc_closest_center(data_array, center_array):
    """Takes in two arrays, normalizes them, and calculates
    the Euclidean distance of points in each array to each other."""
    #Calculate euclidean distance of data pts to k_clusters
    dist_matrix= calc_dist_two_arrays(data_array,center_array)
    assigned_center= dist_matrix.idxmin(axis=1)
    
    #Return label of closest center
    return assigned_center, dist_matrix 

In [428]:
def calc_cluster_variation(k_labels, labels_list, data_array):
    
    labels_copy= labels_list.copy().reset_index()
    print(labels_copy[0].unique())
    total_cluster_var= []
    
    for k in range(k_labels):
        print(k, len(labels_copy[labels_copy[0]== k]))
        group_data= labels_copy[labels_copy[0]== k].merge(data_array.reset_index(),
                               left_on= 'index', right_on= 'index').set_index('index')
        #print(group_data)
        group_data= group_data.drop([0], axis=1)
        distance_matrix= calc_dist_two_arrays(group_data, group_data)
        cluster_variation= distance_matrix.sum().sum()/len(distance_matrix)
        total_cluster_var.append(cluster_variation)
    
    return total_cluster_var

In [425]:
#Use functions to calculate the distance of points to centers
labels0, distmatr_output= calc_closest_center(data, centers0)
distmatr_output.head()

,0,1,2,3,4
0,1.864423,1.668980,2.623420,2.306121,1.871721
1,1.570992,1.701405,2.629597,2.048030,1.525177
2,1.822563,2.174674,2.449558,2.112886,2.106961
3,2.272214,2.329365,2.506291,1.951171,1.604222
4,2.649069,2.389747,2.589726,2.399975,2.070168


In [426]:
labels0.unique()

array([1, 4, 0, 3, 2])

## Update the Centers and calculate Within Cluster Variation


Now that we have labels, we can recalculate the coordinates of the centers by taking the mean of the feature values of the members belonging to that current center.

Here we will also calculate the <strong>within cluster variation</strong>, which will be used to trigger the stop of the algorithm when the assigned cluster labels no longer change. 

In [405]:
def update_centers(input_data, k_num, center_labels):
    """This function will take the raw data, current cluster labels, and k value,
    to calculate (for every cluster):
    1) New cluster center means (eg. coordinates)
    2) The cluster variation score to optimize for at every iteration."""
    
    data_copy= input_data.copy()
    labels_copy= center_labels.copy().reset_index()
    
    cluster_data_list= []
    center_means_list= []
    clust_var_list= []
    
    for k in range(k_num):
        #First group the original data points based on their assigned cluster label
        cluster_data= labels_copy[labels_copy[0]== k].merge(data_copy.reset_index(),
                               left_on= 'index', right_on= 'index').set_index('index')
        cluster_data= cluster_data.drop([0], axis=1)
        cluster_data_list.append(cluster_data)
        
        #Now calculate within cluster variation (sum of euclidean distances/# of points)
        #by calculating the euc. distance of points to each other
        within_distances= calc_dist_two_arrays(cluster_data, cluster_data)
        clust_var= within_distances.sum().sum()/len(within_distances)
        clust_var_list.append(clust_var)
        
        #Now calculate the new mean center from the subset of points belonginng to this cluster
        new_center= cluster_data.mean(axis=0).to_frame().T
        center_means_list.append(new_center)
        
                
    #Output list of new center means, list of cluster variation scores for each cluster
    return clust_var_list, center_means_list

In [406]:
#Seems to be working but this error is confusing
cluster_opt= update_centers(data, 5, labels0)

In [407]:
cluster_opt[0]

[2472.0926023703273,
 4879.3902535343977,
 8394.7902389179126,
 441.30431927347297,
 6044.2441466715918]

## Putting it all together

Given some data, KMeans tries to find the most natural groups/clusters for this data based on their distance to K cluster centers. The algorithm updates the positions of these cluster centers iteratively by taking the mean of the members' positions until there is no change in the positions of these centers, and these groups are defined as the final assignments. 

The steps of operation can be defined as follows:

1. Define number of cluster center, K
2. Define initial K cluster centers, by selecting random values in the feature space.
3. Calculate the distance of all points to the cluster centers, and assign point to the nearest center.
4. Repeat: 

    a) Update the position of the cluster centers by taking the mean of their feature values of it's community members.
    
    b) Recalculate the distance between all data points and the new centers. 
    
    c) Stop when assignments/labels no longer change.
    
There are different ways to perform these operations, with multiple variations of the above. Here, I will calculate the minimal example for simplicity.

We'll also need data to sample, so to use an interesting one, let's use the CDC Nutrition Demographic data uploaded to [Kaggle](https://www.kaggle.com/cdc/national-health-and-nutrition-examination-survey). 

In [431]:
labels0.values

array([4, 4, 4, ..., 2, 1, 1])

In [429]:
centers0= pd.DataFrame(initial_centers(5, data), columns= data.columns)
labels0, distmatr_output= calc_closest_center(data, centers0)

cluster_var_score_0= calc_cluster_variation(5, labels0, centers0)
        
#variation_score= cluster_var_score_0[1]
#var_diff= variation_score

#while var_diff!= 0:
#     #Update the centers
#     variation, new_clust_means= update_centers(data, 5, labels0)
#     var_diff= variation_score-new_variation
#     variation_score= new_variation
    
#     cluster_var_score= calc_cluster_variation(5, labels0, centers0).sum()

[4 1 2 0 3]
0 1826


ValueError: Found array with 0 sample(s) (shape=(0, 42)) while a minimum of 1 is required by MinMaxScaler.

In [416]:
labels0.unique()

array([0, 2, 4, 1, 3])

# Under Progress

In [311]:
sample0= [0,1,2,3,4,5,5]
sample1= [0,1,2,3,4,5,5]

In [313]:
labels0

0        3
1        1
2        3
3        1
4        3
5        1
6        3
7        3
8        1
9        3
10       1
11       3
12       3
13       0
14       3
15       0
16       1
17       1
18       1
19       3
20       2
21       2
22       3
23       3
24       1
25       3
26       3
27       3
28       3
29       3
        ..
10145    3
10146    3
10147    3
10148    2
10149    3
10150    2
10151    3
10152    3
10153    4
10154    2
10155    2
10156    4
10157    3
10158    4
10159    3
10160    3
10161    3
10162    4
10163    3
10164    3
10165    1
10166    3
10167    3
10168    4
10169    2
10170    3
10171    3
10172    3
10173    4
10174    4
Length: 10175, dtype: int64